必要なライブラリのインポート

In [40]:
import pandas as pd
import openpyxl
from openpyxl import Workbook
from openpyxl.chart import Reference, LineChart, Series
from openpyxl.utils.dataframe import dataframe_to_rows
from natsort import natsorted
import glob
from openpyxl.chart.shapes import GraphicalProperties
from openpyxl.drawing.line import LineProperties
from openpyxl.drawing.text import Paragraph, ParagraphProperties, CharacterProperties, RichTextProperties, Font
from openpyxl.chart.text import RichText
from openpyxl.utils.units import pixels_to_EMU
from openpyxl.chart.legend import Legend
from openpyxl.chart.layout import Layout, ManualLayout
import os
from pathlib import Path
from datetime import datetime
import win32com.client
import subprocess

グラフ凡例リスト,読み込む`csv`データリスト

In [41]:
wb = Workbook()
ws = wb.active
# 凡例に使うリスト
# 距離比較
path_list2_origin = natsorted(glob.glob("x*"))
path_list2 = [path.replace("x", "x=") for path in path_list2_origin]

# 温度比較
path_list3 = []
for f in natsorted(glob.glob('**/**/**/t*')):
    path_list3.append(os.path.split(f)[1])

# 会話or咳比較
path_list4 = []
for f in natsorted(glob.glob('**/**/c*')):
    path_list4.append(os.path.split(f)[1])

# 口呼吸or鼻呼吸比較
path_list5 = []
for f in natsorted(glob.glob('x500/co*')):
    path_list5.append(os.path.split(f)[1])

#首角度比較
path_list6 = []
for f in natsorted(glob.glob('x500/open_noses/cough/**/theta_*phi_*')):
    path_list6_origin = os.path.split(f)[1]
    path_list6.append(path_list6_origin.replace("theta_", "θ=").replace("phi_", ",φ="))

#格子比較(感染者側の口周り格子)
path_list7 = []
for f in natsorted(glob.glob('**/**/**/**/**/lattice_*')):
    path_list7.append(os.path.split(f)[1])

読み込む`csv`リスト,分類に使うリストをここに保存.(ここのみ編集する)

In [42]:
path_list1 = natsorted(glob.glob("x500/open_noses/cough/t20/theta_*phi_*/*.csv"))
series_list = path_list6

`csv`データ抽出,グラフ用データ追記

In [43]:
if series_list == path_list2:
    num_dataset = 3
elif series_list == path_list6:
    num_dataset = 5

for index in range(0,num_dataset):
    df = pd.read_csv(path_list1[index])
    df["volume[×1e-9ml]"] = df["volume[ml]"]*1e9
    Data = df[["num_drop","volume[×1e-9ml]"]]
    
    #ここまでデータの抽出
    ID = []
    ID.extend(str(i) for i in range(1,54))
    ID.extend(['LeftUp','LeftLow','RightUp','RightMid','RightLow'])
    #leftup : 2 ~ 12
    #leftlow : 13 ~ 24
    #rightup : 25 ~ 35
    #rightmid : 36 ~ 39
    #rightlow : 40 ~ 54
    Data_c = Data.copy()
    Data_c.loc["leftup"] = Data_c.iloc[0:11,:].sum()
    Data_c.loc["leftlow"] = Data_c.iloc[11:23,:].sum()
    Data_c.loc["rightup"] = Data_c.iloc[23:34,:].sum()
    Data_c.loc["rightmid"] = Data_c.iloc[34:38,:].sum()
    Data_c.loc["rightlow"] = Data_c.iloc[38:53,:].sum()
    Data_c.insert(loc = 0, column = 'ID', value = ID)
    outputData = Data_c.iloc[53:58]

    for i in dataframe_to_rows(outputData,index=None,header=True):
        ws.append(i)

Chart1(飛沫数)基本設定

In [44]:
#c1
c1 = LineChart()

c1.y_axis.title = "number of droplets"
c1.x_axis.title = "Airway site"
c1_data_row_min_list = [2,8,14,20,26]
c1_data_row_max_list = [6,12,18,24,30]
for index in range(0,num_dataset):
    data = Reference(ws,min_col = 2,max_col=2, min_row=c1_data_row_min_list[index],max_row=c1_data_row_max_list[index])
    
    category = Reference(ws, min_col = 1, max_col=1, min_row=2, max_row= 6)
    series = Series(data, title=series_list[index])
    c1.append(series)
    c1.set_categories(category)

Chart1オプション設定

In [45]:
#option
#ser1
ser1=c1.series[0]

ser1.marker.symbol = 'circle'
ser1.marker.size = 8
ser1.marker.graphicalProperties.solidFill = 'FF0000'
ser1.marker.graphicalProperties.line.solidFill = 'FF0000'
ser1.graphicalProperties.line.solidFill = 'FF0000'
ser1.graphicalProperties.line.width = pixels_to_EMU(1.0)

#ser2
ser2=c1.series[1]

ser2.marker.symbol = 'diamond'
ser2.marker.size = 8
ser2.marker.graphicalProperties.solidFill = '008000'
ser2.marker.graphicalProperties.line.solidFill = '008000'
ser2.graphicalProperties.line.solidFill = '008000'
ser2.graphicalProperties.line.width = pixels_to_EMU(1.0)

# #ser3
ser3=c1.series[2]

ser3.marker.symbol = 'square'
ser3.marker.size = 8
ser3.marker.graphicalProperties.solidFill = '0000FF'       
ser3.marker.graphicalProperties.line.solidFill = '0000FF'
ser3.graphicalProperties.line.solidFill = '0000FF'
ser3.graphicalProperties.line.width = pixels_to_EMU(1.0)

if num_dataset == 5:
    # #ser4
    ser4=c1.series[3]

    ser4.marker.symbol = 'circle'
    ser4.marker.size = 8
    ser4.marker.graphicalProperties.solidFill = '000000'       
    ser4.marker.graphicalProperties.line.solidFill = '000000'
    ser4.graphicalProperties.line.solidFill = '000000'
    ser4.graphicalProperties.line.width = pixels_to_EMU(1.0)

    # #ser5
    ser5=c1.series[4]

    ser5.marker.symbol = 'diamond'
    ser5.marker.size = 8
    ser5.marker.graphicalProperties.solidFill = '00FF00'       
    ser5.marker.graphicalProperties.line.solidFill = '00FF00'
    ser5.graphicalProperties.line.solidFill = '00FF00'
    ser5.graphicalProperties.line.width = pixels_to_EMU(1.0)

#axis
c1.x_axis.majorTickMark = "in"
c1.y_axis.majorTickMark = "in"
c1.y_axis.majorGridlines = None
c1.x_axis.majorGridlines = None
c1.y_axis.min = 0
c1.graphical_properties = GraphicalProperties(ln=LineProperties(noFill=True))
c1.plot_area.spPr = GraphicalProperties(ln = LineProperties(solidFill = "000000"))
c1.x_axis.spPr = GraphicalProperties(ln =LineProperties(solidFill="000000"))
c1.y_axis.spPr = GraphicalProperties(ln =LineProperties(solidFill="000000"))

#font
font = Font(typeface='Times New Roman')
size = 1000 # 10 point size
char_prop = CharacterProperties(latin=font, sz=size, b=True, solidFill="000000")
para_prop = ParagraphProperties(defRPr=char_prop)
c1.x_axis.title.tx.rich.p[0].pPr = para_prop
c1.y_axis.title.tx.rich.p[0].pPr = para_prop

char_prop = CharacterProperties(latin=font, sz=size, b=False)
para_prop = ParagraphProperties(defRPr=char_prop)
rich_text = RichText(p=[Paragraph(pPr=para_prop, endParaRPr=char_prop)])
c1.legend.txPr = rich_text
c1.x_axis.txPr = rich_text
c1.y_axis.txPr = rich_text

#Layout
c1.layout = Layout(ManualLayout(
    x = 0.05, 
    y = 0.1, 
    h = 0.8, 
    w = 0.9, 
    xMode="edge", 
    yMode="edge"
    ))
c1.legend.layout = Layout(manualLayout = ManualLayout(
        x=-0.4,
        y=0,
        h=0.1,
        w=0.8,
        yMode="edge"
    ))

Chart2(飛沫体積)基本設定

In [46]:
#c2
c2 = LineChart()

c2.y_axis.title = "volume [×1e-9]"
c2.x_axis.title = "Airway site"

c2_data_row_min_list = [2,8,14,20,26]
c2_data_row_max_list = [6,12,18,24,30]
for index in range(0,num_dataset):
    data = Reference(ws,min_col = 3,max_col=3, min_row=c2_data_row_min_list[index],max_row=c2_data_row_max_list[index])

    category = Reference(ws, min_col = 1, max_col=1, min_row=2, max_row= 6)

    series = Series(data, title=series_list[index])
    c2.append(series)
    c2.set_categories(category)

Chart2オプション設定

In [47]:
#option
#ser6
ser6=c2.series[0]

ser6.marker.symbol = 'circle'
ser6.marker.size = 8
ser6.marker.graphicalProperties.solidFill = 'FF0000'
ser6.marker.graphicalProperties.line.solidFill = 'FF0000'
ser6.graphicalProperties.line.solidFill = 'FF0000'
ser6.graphicalProperties.line.width = pixels_to_EMU(1.0)

#ser7
ser7=c2.series[1]

ser7.marker.symbol = 'diamond'
ser7.marker.size = 8
ser7.marker.graphicalProperties.solidFill = '008000'
ser7.marker.graphicalProperties.line.solidFill = '008000'
ser7.graphicalProperties.line.solidFill = '008000'
ser7.graphicalProperties.line.width = pixels_to_EMU(1.0)

# #ser8
ser8=c2.series[2]

ser8.marker.symbol = 'square'
ser8.marker.size = 8
ser8.marker.graphicalProperties.solidFill = '0000FF'
ser8.marker.graphicalProperties.line.solidFill = '0000FF'
ser8.graphicalProperties.line.solidFill = '0000FF'
ser8.graphicalProperties.line.width = pixels_to_EMU(1.0)

if num_dataset == 5:
    # #ser9
    ser9=c2.series[3]

    ser9.marker.symbol = 'circle'
    ser9.marker.size = 8
    ser9.marker.graphicalProperties.solidFill = '000000'       
    ser9.marker.graphicalProperties.line.solidFill = '000000'
    ser9.graphicalProperties.line.solidFill = '000000'
    ser9.graphicalProperties.line.width = pixels_to_EMU(1.0)

    # #ser10
    ser10=c2.series[4]

    ser10.marker.symbol = 'diamond'
    ser10.marker.size = 8
    ser10.marker.graphicalProperties.solidFill = '00FF00'       
    ser10.marker.graphicalProperties.line.solidFill = '00FF00'
    ser10.graphicalProperties.line.solidFill = '00FF00'
    ser10.graphicalProperties.line.width = pixels_to_EMU(1.0)

c2.x_axis.majorTickMark = "in"
c2.y_axis.majorTickMark = "in"
c2.y_axis.majorGridlines = None
c2.x_axis.majorGridlines = None
c2.y_axis.min = 0
c2.graphical_properties = GraphicalProperties(ln=LineProperties(noFill=True))
c2.plot_area.spPr = GraphicalProperties(ln = LineProperties(solidFill = "000000"))
c2.x_axis.spPr = GraphicalProperties(ln =LineProperties(solidFill="000000"))
c2.y_axis.spPr = GraphicalProperties(ln =LineProperties(solidFill="000000"))

#font
font = Font(typeface='Times New Roman')
size = 1000 # 10 point size
char_prop = CharacterProperties(latin=font, sz=size, b=True, solidFill="000000")
para_prop = ParagraphProperties(defRPr=char_prop)
c2.x_axis.title.tx.rich.p[0].pPr = para_prop
c2.y_axis.title.tx.rich.p[0].pPr = para_prop

char_prop = CharacterProperties(latin=font, sz=size, b=False)
para_prop = ParagraphProperties(defRPr=char_prop)
rich_text = RichText(p=[Paragraph(pPr=para_prop, endParaRPr=char_prop)])
c2.legend.txPr = rich_text
c2.x_axis.txPr = rich_text
c2.y_axis.txPr = rich_text

#Layout
c2.layout = Layout(ManualLayout(
    x = 0.05, 
    y = 0.1, 
    h = 0.8, 
    w = 0.9, 
    xMode="edge", 
    yMode="edge"
    ))
c2.legend.layout = Layout(manualLayout = ManualLayout(
        x=-0.4,
        y=0,
        h=0.1,
        w=0.8,
        yMode="edge"
    ))

`result`内に保存用ディレクトリを作る.(初回のみ)

In [ ]:
directory_1 = 'result/distance'
directory_2 = 'result/angle'

if not os.path.exists(directory_1):
    os.makedirs(directory_1)
    print('Directory "distance" has been created')
else:
    print('directory "distance" exists.')

if not os.path.exists(directory_2):
    os.makedirs(directory_2)
    print('Directory "angle" has been created')
else:
    print('directory "angle" exists.')


Chart1,Chart2,`xlsx`ファイル出力設定

In [ ]:
ws.add_chart(c1, "E22")
ws.add_chart(c2,"N22")
# 命名規則,保存ディレクトリの指定
today = datetime.now()
date_string = today.strftime("%Y%m%d")
Read_file_path = Path(path_list1[0])
if series_list == path_list2:
    filename = Read_file_path.parent.name + "_" + date_string + ".xlsx"
    resultfilepath = Path("./result/distance/" + filename)
elif series_list == path_list6:
    filename = Read_file_path.parts[0] + "_" + date_string + ".xlsx"
    resultfilepath = Path("./result/angle/" + filename)

wb.save(resultfilepath)
print(filename + " was generated.")

グラフ画像保存(excelからグラフを抽出して,画像として保存)

In [50]:
excel = win32com.client.Dispatch("Excel.Application")
excel.Visible = False

filepath = r"C:/Users/b1123/document/Lab/Program/droplets_simulation/BoxCount_ICCS2025/" + str(resultfilepath)
directorypath = r"C:/Users/b1123/document/Lab/Program/droplets_simulation/BoxCount_ICCS2025/"
workbook = excel.Workbooks.Open(filepath)
sheet = workbook.Sheets(1)

chart1 = sheet.ChartObjects(1).Chart
chart2 = sheet.ChartObjects(2).Chart

if series_list == path_list2:
    imagename_number = Read_file_path.parent.name + "_" + date_string + ".png"
    imagename_volume = Read_file_path.parent.name + "_" + date_string + ".png"
    chart1.Export(r"C:/Users/b1123/document/Lab/Data/droplets_count_ICCS/Graph/distance/number/" + imagename_number, "PNG")
    chart2.Export(r"C:/Users/b1123/document/Lab/Data/droplets_count_ICCS/Graph/distance/volume/" + imagename_volume, "PNG")
    figurepath = r"C:/Users/b1123/document/Lab/Data/droplets_count_ICCS/Graph/distance/"
elif series_list == path_list6:
    imagename_number = Read_file_path.parts[0] + "_" + date_string + ".png"
    imagename_volume = Read_file_path.parts[0] + "_" + date_string + ".png"
    chart1.Export(r"C:/Users/b1123/document/Lab/Data/droplets_count_ICCS/Graph/angle/number/" + imagename_number, "PNG")
    chart2.Export(r"C:/Users/b1123/document/Lab/Data/droplets_count_ICCS/Graph/angle/volume/" + imagename_volume, "PNG")
    figurepath = r"C:/Users/b1123/document/Lab/Data/droplets_count_ICCS/Graph/angle/"

workbook.Close(SaveChanges=False)
excel.Quit()

保存先ディレクトリを開く

In [ ]:
subprocess.Popen(['start', directorypath], shell=True)
subprocess.Popen(['start', figurepath], shell=True)